In [1]:
using ParallelTemperingMonteCarlo

┌ Warning: Package ParallelTemperingMonteCarlo does not have Graphs in its dependencies:
│ - If you have ParallelTemperingMonteCarlo checked out for development and have
│   added Graphs as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with ParallelTemperingMonteCarlo
└ Loading Graphs into ParallelTemperingMonteCarlo from project dependency, future warnings for ParallelTemperingMonteCarlo are suppressed.


In [2]:
using Random
using Plots


(process:43744): GLib-GIO-WARNING **: 10:57:47.796: Unexpectedly, UWP app `Microsoft.ScreenSketch_11.2305.26.0_x64__8wekyb3d8bbwe' (AUMId `Microsoft.ScreenSketch_8wekyb3d8bbwe!App') supports 29 extensions but has no verbs

(process:43744): GLib-GIO-WARNING **: 10:57:47.927: Unexpectedly, UWP app `Clipchamp.Clipchamp_2.7.9.0_neutral__yxz26nhyzhsrt' (AUMId `Clipchamp.Clipchamp_yxz26nhyzhsrt!App') supports 41 extensions but has no verbs


In [3]:
#set random seed - for reproducibility
Random.seed!(1234)

# number of atoms
n_atoms = 13

# temperature grid
ti = 5.
tf = 30.
n_traj = 32

temp = TempGrid{n_traj}(ti,tf) 

# MC simulation details

mc_cycles = 1000000 #default 20% equilibration cycles on top


mc_sample = 1  #sample every mc_sample MC cycles

#move_atom=AtomMove(n_atoms) #move strategy (here only atom moves, n_atoms per MC cycle)
displ_atom = 0.1 # Angstrom
n_adjust = 100

max_displ_atom = [0.1*sqrt(displ_atom*temp.t_grid[i]) for i in 1:n_traj]

mc_params = MCParams(mc_cycles, n_traj, n_atoms, mc_sample = mc_sample, n_adjust = n_adjust)

#moves - allowed at present: atom, volume and rotation moves (volume,rotation not yet implemented)
move_strat = MoveStrategy(atom_moves = n_atoms)  

#ensemble
ensemble = NVT(n_atoms)

#ELJpotential for neon
#c1=[-10.5097942564988, 0., 989.725135614556, 0., -101383.865938807, 0., 3918846.12841668, 0., -56234083.4334278, 0., 288738837.441765]
#elj_ne1 = ELJPotential{11}(c1)

c=[-10.5097942564988, 989.725135614556, -101383.865938807, 3918846.12841668, -56234083.4334278, 288738837.441765]
pot = ELJPotentialEven{6}(c)

#starting configurations
#icosahedral ground state of Ne13 (from Cambridge cluster database) in Angstrom
pos_ne13 = [[2.825384495892464, 0.928562467914040, 0.505520149314310],
[2.023342172678102,	-2.136126268595355, 0.666071287554958],
[2.033761811732818,	-0.643989413759464, -2.133000349161121],
[0.979777205108572,	2.312002562803556, -1.671909307631893],
[0.962914279874254,	-0.102326586625353, 2.857083360096907],
[0.317957619634043,	2.646768968413408, 1.412132053672896],
[-2.825388342924982, -0.928563755928189, -0.505520471387560],
[-0.317955944853142, -2.646769840660271, -1.412131825293682],
[-0.979776174195320, -2.312003751825495, 1.671909138648006],
[-0.962916072888105, 0.102326392265998,	-2.857083272537599],
[-2.023340541398004, 2.136128558801072,	-0.666071089291685],
[-2.033762834001679, 0.643989905095452, 2.132999911364582],
[0.000002325340981,	0.000000762100600, 0.000000414930733]]

#convert to Bohr
AtoBohr = 1.8897259886
pos_ne13 = pos_ne13 * AtoBohr

length(pos_ne13) == n_atoms || error("number of atoms and positions not the same - check starting config")


true

In [5]:
#boundary conditions 
bc_ne13 = SphericalBC(radius=5.32*AtoBohr)   #5.32 Angstrom

#starting configuration
start_config = Config(pos_ne13, bc_ne13)

#histogram information
n_bin = 100
#en_min = -0.006    #might want to update after equilibration run if generated on the fly
#en_max = -0.001    #otherwise will be determined after run as min/max of sampled energies (ham vector)

#construct array of MCState (for each temperature)
mc_states_s = [MCState(temp.t_grid[i], temp.beta_grid[i], start_config, pot) for i in 1:n_traj]

#results = Output(n_bin, max_displ_vec)
results_s = Output{Float64}(n_bin; en_min = mc_states_s[1].en_tot)

@time ptmc_run!((mc_states_s, move_strat, mc_params, pot, ensemble, results_s); save=true)

equilibration done


MC loop done.


[0.0003267794193287531, 0.00034846940029610975, 0.0003784736083017704, 0.00040938007467021195, 0.0004413170776657435, 0.0004764133068330665, 0.0005226002760656093, 0.0005737193788054905, 0.0006266721057906895, 0.0006879925637270109, 0.0008400937676724244, 0.0009627215845335636, 0.001287264579770501, 0.0016218136880412173, 0.002373388573191344, 0.003046971846610148, 0.0032953664921848646, 0.00306389173492443, 0.0025893346807044373, 0.002251203530704531, 0.001976607898042438, 0.001888019523308829, 0.0018420662294167422, 0.0017563336060393423, 0.0017628032120420806, 0.0017372391966938587, 0.0017081808634770057, 0.001664611524555201, 0.0016113404170340119, 0.0015587679998746167, 0.0015130989681428674, 0.0014284559170118349]
done
426.559982 seconds (1.13 G allocations: 170.991 GiB, 5.12% gc time, 0.41% compilation time)


In [6]:
#boundary conditions 
bc_ne13 = init_AdjacencyBC(pos_ne13, 4.39*AtoBohr)  #4.39 Angstrom

#starting configuration
start_config = Config(pos_ne13, bc_ne13)

#histogram information
n_bin = 100
#en_min = -0.006    #might want to update after equilibration run if generated on the fly
#en_max = -0.001    #otherwise will be determined after run as min/max of sampled energies (ham vector)

#construct array of MCState (for each temperature)
mc_states_a = [MCState(temp.t_grid[i], temp.beta_grid[i], start_config, pot) for i in 1:n_traj]

#results = Output(n_bin, max_displ_vec)
results_a = Output{Float64}(n_bin; en_min = mc_states_a[1].en_tot)

@time ptmc_run!((mc_states_a, move_strat, mc_params, pot, ensemble, results_a); save=true)

equilibration done
MC loop done.
[0.00032947196589893995, 0.000353596338221599, 0.00038081878204544554, 0.0004119591923848686, 0.00044594984458779126, 0.0004890799509855542, 0.0005321428817624851, 0.0005945545161521544, 0.0006407926702728472, 0.0007375233204157946, 0.0008544512048058332, 0.0011091134461762465, 0.001635880704821368, 0.002481277467756681, 0.0032439546071898516, 0.0035120452264243273, 0.00315446117074591, 0.002600275975711276, 0.002091830601620603, 0.001857310246545879, 0.001699889888785128, 0.0016997943087504947, 0.0016473117561359256, 0.0016521121782435743, 0.0016480492872931235, 0.0016355580344714626, 0.0016054109620785382, 0.0015569789994398658, 0.0015520141325738417, 0.001506271960398316, 0.0014265751537577047, 0.0013538631422740613]
done


7561.782470 seconds (26.26 G allocations: 3.007 TiB, 8.36% gc time, 0.02% compilation time)


In [11]:
plot([multihistogram(results_s,temp), (multihistogram(results_a,temp))], label = ["spherical boundary condition" "adjacency boundary condition"], legend = true, xlabel="Temperature", ylabel="Heat Capacity")
png("mylove")

B Vector Calculated
A Matrix Calculated
system solved!
Entropy Found


Quantities found
analysis complete
B Vector Calculated
A Matrix Calculated
system solved!
Entropy Found


Quantities found
analysis complete